# Develop N-impulse problem

In [1]:
using DifferentialEquations
using Plots
using LinearAlgebra
import ForwardDiff
import DiffResults
using AstrodynamicsBase
import joptimise
using Printf

In [2]:
plotly()

Plots.PlotlyBackend()

In [33]:
include("../../julia-r3bp/R3BP/src/R3BP.jl")
include("../src/SailorMoon.jl")   # relative path to main file of module

Main.SailorMoon

In [34]:
param3b = SailorMoon.dyanmics_parameters()
lps = SailorMoon.lagrange_points(param3b.mu2)

5×6 Matrix{Float64}:
  0.836915   0.0       0.0  0.0  0.0  0.0
  1.15568    0.0       0.0  0.0  0.0  0.0
 -1.00506    0.0       0.0  0.0  0.0  0.0
  0.487849   0.866025  0.0  0.0  0.0  0.0
  0.487849  -0.866025  0.0  0.0  0.0  0.0

In [35]:
lp = 2
Az_km = 1200.0
println("Halo guess Az_km: $Az_km")
northsouth = 3   # 1 or 3
guess0 = R3BP.halo_analytical_construct(param3b.mu2, lp, Az_km, param3b.lstar, northsouth)
res = R3BP.ssdc_periodic_xzplane([param3b.mu2,], guess0.x0, guess0.period, fix="period")
res.flag

Halo guess Az_km: 1200.0


1

In [36]:
x0_stm = vcat(res.x0, reshape(I(6), (6^2,)))[:]
prob_cr3bp_stm = ODEProblem(R3BP.rhs_cr3bp_svstm!, x0_stm, res.period, (param3b.mu2))
sol = solve(prob_cr3bp_stm, Tsit5(), reltol=1e-12, abstol=1e-12)#, saveat=LinRange(0, period, n+1))
monodromy = R3BP.get_stm(sol, 6)   # get monodromy matrix
ys0 = R3BP.get_eigenvector(monodromy, true, 1);

Linear stability ν = 618.7618470919092


In [38]:
# arrival LPO object
LPOArrival = SailorMoon.CR3BPLPO(
    res.x0, res.period, ys0, prob_cr3bp_stm, 1e-6, Tsit5(), 1e-12, 1e-12, 0.005
);

In [39]:
tmax_si = 0.3  # N
isp_si = 3500  # sec
mdot_si = tmax_si / (isp_si * 9.81)
mstar = 2000  # kg

tmax = AstrodynamicsBase.dimensional2canonical_thrust(
    tmax_si, mstar, param3b.lstar, param3b.tstar
)
mdot = AstrodynamicsBase.dimensional2canonical_mdot(
    mdot_si, mstar, param3b.tstar
)

0.0016413295927890758

### Construct optimization problem

In [40]:
params = [param3b.mu2, param3b.mus, 0.0, param3b.as, param3b.oms, 0.0, 0.0, 0.0, 0.0, 0.0]
_prob_base = ODEProblem(R3BP.rhs_bcr4bp_thrust!, rand(7), [0,1], params)

ODEProblem with uType Vector{Float64} and tType Int64. In-place: true
timespan: (0, 1)
u0: 7-element Vector{Float64}:
 0.42722981104758606
 0.07310855684974915
 0.9147722293230062
 0.9179726210112293
 0.46444074441507
 0.5527229458361733
 0.4241552121795752

In [41]:
propagate_trajectory = function (x::AbstractVector{T}, get_sols::Bool=false) where T
    # unpack
    nx = length(x)
    θf, tof, eta, sma, ecc, raan, ϕ = x[1:7]  # θf: Sun angle at final time
    tau1     = x[8 : floor(Int, (nx-7)/2 + 7)]  # discretization numbers are the same for the first & second arc
    tau2     = x[floor(Int, (nx-7)/2 + 8) : end]
    tof_fwd = tof * eta
    tof_bck = tof * (1 - eta)
    
    # construct initial state
    sv0_kep = [sma, ecc, 0.0, raan, 0.0, 0.0]
    θ0 = θf - param3b.oms*(tof_fwd + tof_bck)   # initial Sun angle
    sv0_i = AstrodynamicsBase.kep2cart(sv0_kep, param3b.mu1)
    sv0 = vcat(inertial2rotating(sv0_i, θ0, 1.0) + [-param3b.mu2,0,0,0,0,0], 1.0)
    
    # construct final state
    #x0_stm = vcat(LPOArrival.x0, reshape(I(6), (36,)))
    #tspan = [0, ϕ*LPOArrival.period]
    #prob_cr3bp_stm = ODEProblem(R3BP.rhs_cr3bp_svstm!, x0_stm, tspan, [param3b.mu2,])
    #sol = solve(prob_cr3bp_stm, Tsit5(), reltol=1e-12, abstol=1e-12)
    svf = vcat(SailorMoon.set_terminal_state(ϕ, param3b, LPOArrival), 1.0)
    
    # initialize storage
    sols_fwd, sols_bck = [], []
    params_fwd, params_bck = [], []
    
    # forward propagation
    nhalf = Int(n/2)
    tspan_fwd = [0, tof_fwd/nhalf]
    for i = 1:nhalf
        τ, γ, β = tau1[3*i-2 : 3*i]
        params = [param3b.mu2, param3b.mus, θ0, param3b.as, param3b.oms, τ, γ, β, mdot, tmax]
        _prob = remake(
            _prob_base; tspan=tspan_fwd, 
            u0 = sv0,
            p = params,
        )
        sol = DifferentialEquations.solve(_prob, Tsit5(), reltol=1e-12, abstol=1e-12)
        if get_sols
            push!(sols_fwd, sol)
            push!(params_fwd, params)
        end
        # update θ0 and sv0
        θ0 += param3b.oms*sol.t[end]
        sv0 = sol.u[end]
    end
    
    # back propagation
    tspan_bck = [0, -tof_bck/(n-nhalf)]
    for i = 1:n-nhalf
        τ, γ, β = tau2[3*i-2 : 3*i]
        params = [param3b.mu2, param3b.mus, θf, param3b.as, param3b.oms, τ, γ, β, mdot, tmax]
        _prob = remake(
            _prob_base; tspan=tspan_bck, 
            u0 = svf,
            p = params,
        )
        sol = DifferentialEquations.solve(_prob, Tsit5(), reltol=1e-12, abstol=1e-12)
        if get_sols
            push!(sols_bck, sol)
            push!(params_bck, params)
        end
        # update θf (note sol.t[end] is negative so += is correct) and svf
        θf += param3b.oms*sol.t[end]
        svf = sol.u[end]
    end
    
    # residual
    if get_sols == false
        return svf - sv0
    else
        return sols_fwd, sols_bck, params_fwd, params_bck
    end
end

#9 (generic function with 2 methods)

In [42]:
n = 30

30

In [43]:
bounds_tau = [0,1]
bounds_γ   = [-π, π]
bounds_β   = [-π, π]
# θf, tof, eta, sma, ecc, raan = x
xtest = [
    2.4, 22, 0.4, 1.8, 0.87, 4.5, 0.02
]
for i = 1:n
    xtest = vcat(xtest, [0.1,0,0])
end

lx = [
    2.6, 18, 0.3, 1.9, 0.7, 0.0, -1.0
]
ux = [
    3.2, 27, 0.7, 2.4, 0.995, 2π, 1.0
]
for i = 1:n
    lx = vcat(lx, [0,bounds_γ[1],bounds_β[1]])
    ux = vcat(ux, [0,bounds_γ[1],bounds_β[2]])
end

In [69]:
sols_fwd, sols_bck, params_fwd, params_bck = propagate_trajectory(xtest, true);
#propagate_trajectory(xtest, false)

In [45]:
cs_fwd = palette([:darkred, :navyblue], max(Int(n/2),2))
cs_bck = palette([:darkgreen, :goldenrod], max(n-Int(n/2),2))

pcart = plot(size=(700,500), frame_style=:box, aspect_ratio=:equal, grid=0.4)
scatter!(pcart, lps[:,1], lps[:,2], marker=:diamond, color=:red, label="LPs")
# trajectory
for (ifwd,sol_fwd) in enumerate(sols_fwd)
    plot!(pcart, Array(sol_fwd)[1,:], Array(sol_fwd)[2,:], color=cs_fwd[ifwd], 
        linewidth=1.5, label="fwd $ifwd", linestyle=:dashdot)
end
for (ibck,sol_bck) in enumerate(sols_bck)
    plot!(pcart, Array(sol_bck)[1,:], Array(sol_bck)[2,:], color=cs_bck[ibck], 
        linewidth=1.5, label="bck $ibck", linestyle=:solid)
end
plot!(pcart; title="Initial guess")
pcart

In [46]:
# problem settings
ng = 3
lg = [0.0 for idx=1:ng];
ug = [0.0 for idx=1:ng];

fitness! = function (g, x)
    # evaluate objective & objective gradient (trivial)
    #f = 1       # whichever x corresponds to e.g. mass at LEO
    #g[:] = propagate_trajectory(x, false)
    
    sols_fwd, sols_bck = propagate_trajectory(x, true)
    sol_fwd = sols_fwd[end]
    sol_bck = sols_bck[end]
    g[:] = sol_bck.u[end][1:3] - sol_fwd.u[end][1:3]
    f = norm(sol_bck.u[end][4:6] - sol_fwd.u[end][4:6])
    return f
end

#15 (generic function with 1 method)

In [47]:
gfoo  = zeros(ng)
fitness!(gfoo, xtest)
gfoo

3-element Vector{Float64}:
 -1.1244217674504409
  0.8570015531246731
 -4.724369966663974e-26

In [48]:
ip_options = Dict(
    "max_iter" => 100,   # approx 100
    "print_level" => 4,
    "acceptable_tol" => 1e-5,
    "constr_viol_tol" => 1e-5,
)
x0 = [el for el in xtest]
xopt, fopt, info = joptimise.minimize(
    fitness!, x0, ng;
    lx=lx, ux=ux, lg=lg, ug=ug,
    #derivatives=joptimise.ForwardAD(),
    options=ip_options,
);

Total number of variables............................:       37
                     variables with only lower bounds:        0
                variables with lower and upper bounds:       37
                     variables with only upper bounds:        0
Total number of equality constraints.................:        3
Total number of inequality constraints...............:        0
        inequality constraints with only lower bounds:        0
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        0


Number of Iterations....: 92

                                   (scaled)                 (unscaled)
Objective...............:   1.0495425543146414e-01    1.0495425543146414e-01
Dual infeasibility......:   1.0234026566049464e-05    1.0234026566049464e-05
Constraint violation....:   8.9873442021826122e-11    8.9873442021826122e-11
Complementarity.........:   1.0000085703611931e-11    1.0000085703611931e-11
Overall NLP e

In [54]:
info

:Solve_Succeeded

In [55]:
sols_fwd, sols_bck, params_fwd, params_bck = propagate_trajectory(xopt, true);
propagate_trajectory(xopt, false)

7-element Vector{Float64}:
 -3.623250921513943e-7
 -2.715376492545829e-7
 -6.39311706946735e-32
  0.014089427997439508
 -0.1040038584905223
 -2.8775313587431643e-26
  0.0

In [56]:
dr = sols_bck[end].u[end][1:3] - sols_fwd[end].u[end][1:3]

3-element Vector{Float64}:
 -3.623250921513943e-7
 -2.715376492545829e-7
 -6.39311706946735e-32

In [57]:
dv = sols_bck[end].u[end][4:6] - sols_fwd[end].u[end][4:6]

3-element Vector{Float64}:
  0.014089427997439508
 -0.1040038584905223
 -2.8775313587431643e-26

In [58]:
norm(dv)*param3b.lstar/param3b.tstar  # km/s

0.1074814693355364

In [59]:
# for i = 1:length(xopt)
#     @printf("% 3.3f | % 3.3f | % 3.3f\n", lx[i], xopt[i], ux[i])
# end

In [60]:
cs_fwd = palette([:darkred, :navyblue], max(Int(n/2),2))
cs_bck = palette([:darkgreen, :goldenrod], max(n-Int(n/2),2))

pcart = plot(size=(700,500), frame_style=:box, aspect_ratio=:equal, grid=0.4)
scatter!(pcart, lps[:,1], lps[:,2], marker=:diamond, color=:red, label="LPs")
# trajectory
for (ifwd,sol_fwd) in enumerate(sols_fwd)
    plot!(pcart, Array(sol_fwd)[1,:], Array(sol_fwd)[2,:], color=cs_fwd[ifwd], 
        linewidth=1.5, label="fwd $ifwd", linestyle=:dashdot)
end
for (ibck,sol_bck) in enumerate(sols_bck)
    plot!(pcart, Array(sol_bck)[1,:], Array(sol_bck)[2,:], color=cs_bck[ibck], 
        linewidth=1.5, label="bck $ibck", linestyle=:solid)
end
plot!(pcart; title="Solved")
pcart

In [61]:
# unpack optimal solution
βf, tof, eta, sma, ecc, raan, ϕ = xopt[1:7]
tof_fwd = tof * eta
tof_bck = tof * (1 - eta)
β0 = βf - param3b.oms*tof                # initial Sun angle

23.9360852656848

## Plot in inertial frame

In [62]:
sols_fwd_i = []
for (ifwd,sol_fwd) in enumerate(sols_fwd)
    _, _, theta0, _, _, _, _, _, _, _ = params_fwd[ifwd]
    sol_fwd_i = zeros(7,length(sol_fwd.u))
    for (idx,sv) in enumerate(sol_fwd.u)
        theta = theta0 + param3b.oml * sol_fwd.t[idx]
        sol_fwd_i[1:6,idx] = rotating2inertial(sv[1:6], theta, param3b.oml)
        sol_fwd_i[7,idx] = sv[7]
    end
    push!(sols_fwd_i, sol_fwd_i)
end
    
sols_bck_i = []
for (ibck,sol_bck) in enumerate(sols_bck)
    _, _, theta0, _, _, _, _, _, _, _ = params_bck[ibck]
    sol_bck_i = zeros(7,length(sol_bck.u))
    for (idx,sv) in enumerate(sol_bck.u)
        theta = theta0 + param3b.oml * sol_bck.t[idx]
        sol_bck_i[1:6,idx] = rotating2inertial(sv[1:6], theta, param3b.oml)
        sol_bck_i[7,idx] = sv[7]
    end
    push!(sols_bck_i, sol_bck_i)
end

In [63]:
cs_fwd = palette([:darkred, :navyblue], max(Int(n/2),2))
cs_bck = palette([:darkgreen, :goldenrod], max(n-Int(n/2),2))

pcart = plot(size=(700,500), frame_style=:box, aspect_ratio=:equal, grid=0.4)
# trajectory
for (ifwd,sol_fwd) in enumerate(sols_fwd_i)
    plot!(pcart, Array(sol_fwd)[1,:], Array(sol_fwd)[2,:], color=cs_fwd[ifwd], 
        linewidth=1.5, label="fwd $ifwd", linestyle=:dashdot)
end
for (ibck,sol_bck) in enumerate(sols_bck_i)
    plot!(pcart, Array(sol_bck)[1,:], Array(sol_bck)[2,:], color=cs_bck[ibck], 
        linewidth=1.5, label="bck $ibck", linestyle=:solid)
end
plot!(pcart; title="Solved")
pcart

## Plot in SE-rotating frame

In [64]:
SE_system = R3BP.get_cr3bp_param(10,399)
EM_system = R3BP.get_cr3bp_param(399,301)

scale_l = EM_system.lstar/SE_system.lstar
scale_v = (EM_system.lstar/EM_system.tstar) / (SE_system.lstar/SE_system.tstar)
scale_t = EM_system.tstar/SE_system.tstar

0.07469828599591845

In [65]:
function planetmoon2sunplanet(
    state::Vector, μ_moon::Real, μ_sun::Real, theta::Real, om::Real,
    scale_l::Real, scale_v::Real, scale_t::Real, final_shift::Bool=false
)
    # 1. shift to Planet-centered, planet-moon rotating frame
    state_1 = state + [μ_moon, 0, 0, 0, 0, 0]
    # 2. convert to PLanet-centered, inertial
    state_2 = rotating2inertial(state_1, theta, om)
    # 3. re-scale
    state_3 = vcat(
        state_2[1:3] * scale_l,
        state_2[4:6] * scale_v
    )
    # 4. convert to Planet-centered, Sun-planet rotating frame
    theta_sp = scale_t * theta
    state_4 = inertial2rotating(state_3, theta_sp, om)
    # 5. shift to Sun-planet barycenter centered, Sun-planet rotating frame
    if final_shift == true
        return state_4 + [1 - μ_sun, 0, 0, 0, 0, 0]
    else
        return state_4
    end
end


planetmoon2sunplanet (generic function with 2 methods)

In [66]:
#params = [param3b.mu2, param3b.mus, θ0, param3b.as, param3b.oms, τ, γ, β, mdot, tmax]

In [67]:
sols_bck_se = []
for (ibck, sol_bck) in enumerate(sols_bck)
    sol_bck_se = zeros(7,length(sol_bck.u))
    for (idx,sv) in enumerate(sol_bck.u)
        _, _, theta, _, _, _, _, _, _, _ = params_bck[ibck]
        #theta = βf + sol_bck.t[idx]
        sol_bck_se[1:6,idx] = planetmoon2sunplanet(
            sv[1:6], param3b.mu2, param3b.mus, theta, 1.0, #param3b.oms,
            scale_l, scale_v, scale_t
        )
        sol_bck_se[7,idx] = sv[7]
    end
    push!(sols_bck_se, sol_bck_se)
end

sols_fwd_se = []
for (ifwd,sol_fwd) in enumerate(sols_fwd)
    sol_fwd_se = zeros(7,length(sol_fwd.u))
    for (idx,sv) in enumerate(sol_fwd.u)
        _, _, theta, _, _, _, _, _, _, _ = params_fwd[ifwd]
        #theta = β0 + sol_fwd.t[idx]
        #@+ sol_bck.t[end] - (tof_fwd - sol_fwd.t[idx])
        sol_fwd_se[1:6,idx] = planetmoon2sunplanet(
            sv[1:6], param3b.mu2, param3b.mus, theta, 1.0, #param3b.oms,
            scale_l, scale_v, scale_t
        )
        sol_fwd_se[7,idx] = sv[7]
    end
    push!(sols_fwd_se, sol_fwd_se)
end

In [68]:
cs_fwd = palette([:darkred, :navyblue], max(Int(n/2),2))
cs_bck = palette([:darkgreen, :goldenrod], max(n-Int(n/2),2))

pcart = plot(size=(700,500), frame_style=:box, aspect_ratio=:equal, grid=0.4)
# trajectory
for (ifwd,sol_fwd) in enumerate(sols_fwd_se)
    plot!(pcart, Array(sol_fwd)[1,:], Array(sol_fwd)[2,:], color=cs_fwd[ifwd], 
        linewidth=1.5, label="fwd $ifwd", linestyle=:dashdot)
end
for (ibck,sol_bck) in enumerate(sols_bck_se)
    plot!(pcart, Array(sol_bck)[1,:], Array(sol_bck)[2,:], color=cs_bck[ibck], 
        linewidth=1.5, label="bck $ibck", linestyle=:solid)
end
plot!(pcart; title="Solved")
pcart